# Module 06 -- Building More Intricate Data Visualizations

In the last module we started building some visualizations in order to answer specific substantive questions. In this module we will look at generating more advanced graphics that have annotations, combine several graphics into one, build some interactive graphics, and also do some mapping.

# Combining Plots with `{patchwork}`
Often you have to combine and place multiple graphics into a single canvas. There are a few ways to do this but the easiest way is that offered by the `{patchwork}` package. Let us use the `diamonds` data for this section, a data frame with 53940 rows and 10 variables:

| Variable | Description |
| :-- | :-- |
| price |  price in US dollars (\$326--\$18,823) |
| carat | weight of the diamond (0.2--5.01) |
| cut | quality of the cut (Fair, Good, Very Good, Premium, Ideal) |
| color | diamond colour, from D (best) to J (worst) |
| clarity | a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best)) |
| x | length in mm (0--10.74) |
| y | width in mm (0--58.9) |
| z | depth in mm (0--31.8) |
| depth | total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79) |
| table | width of top of diamond relative to widest point (43--95) |

## The Basics
To combine multiple plots, we need to save `each plot` with a unique name. I am calling them `p1`, `p2`, etc.

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 14)

In [ ]:
library(patchwork)
library(tidyverse)

ggplot() +
  geom_bar(data = diamonds,
           aes(x = cut, fill = cut)
           ) +
  labs(x = "Cut of the Diamond", y = "Frequency") +
  theme(legend.position = "none") -> p1

ggplot() +
  geom_bar(data = diamonds,
           aes(x = color, fill = color)
           ) +
  labs(x = "Color of the Diamond", y = "Frequency") +
  theme(legend.position = "none") -> p2

ggplot() +
  geom_point(data = diamonds,
           aes(x = carat, y = price, color = cut)
           ) +
  labs(x = "Weight of the Diamond", y = "Price of the Diamond", color = "") +
  theme(legend.position = "bottom") -> p3

ggplot() +
  geom_boxplot(data = diamonds,
           aes(x = price, y = clarity, fill = cut)
           ) +
  labs(y = "Clarity of the Diamond", x = "Price of the Diamond", fill = "") +
  theme(legend.position = "bottom") -> p4

p1 + p2 + p3

Notice the default layout here: `p1 + p2 + p3` gives us the plots all in a row. But you may have other plans, for example, to put the scatterplot in a row all its own.

In [ ]:
(p1 + p2) / p3

Now we have `p3` in the second row, all by itself. Note that this was achieved via the `/` operator. What if we used `|` instead?

In [ ]:
p1 | (p2 + p3)

You ended up with two columns, the first containing only `p1` and the second containing `p2` and `p3`. So make a note of the difference between `|` and `/`. For example, note the following setup:

In [ ]:
p1 | (p2 / p3)

What if we wanted to squeeze in the fourth plot?

In [ ]:
(p1 + p2) / (p3 + p4)

## Annotations
Annotations become helpful because you can add omnibus titles and tags for individual plots. For example,

In [ ]:
(p1 + p2) / (p3 + p4) +
  plot_annotation(
  title = 'The surprising truth about diamonds',
  subtitle = 'These plots will reveal untold secrets about one of our beloved data-sets',
  caption = 'Disclaimer: None of these plots are insightful',
  tag_levels = c('a', '1'),
  tag_prefix = 'Fig. ',
  tag_sep = '.',
  tag_suffix = ':'
  ) &
  theme(
    plot.tag.position = c(0, 1),
    plot.tag = element_text(size = 9, hjust = 0, vjust = 0, color = "steelblue")
    )

## Spacing and Sizing
We can also tweak the sizes of individual rows and columns, control the space between plots, and so on. First up, spacing the plots with `plot_spacer()`

In [ ]:
(p1 + plot_spacer() + p2 + plot_spacer() + p3)

Sizing the plots with relative sizes?

In [ ]:
p1 + p2 + p3 + p4 +
  plot_layout(widths = c(2, 1))

Alternatively, we could specify size with unit vectors, as shown below.

In [ ]:
p1 + p2 + p3 + p4 +
  plot_layout(
    widths = c(2, 1),
    heights = unit(c(5, 1), c('cm', 'null'))
    )

## Moving Beyond the grid

We can use a `layout` design to get a little more flexibility but still retain full control over the result. Layout designs can be done in two ways so let us see the easiest route -- as a text setup. "When using the textual representation it is your responsibility to make sure that each area is rectangular. The only exception is # which denotes empty areas and can thus be of any shape."

In [ ]:
layout <- "
##BBBB
AACCDD
##CCDD
"
p2 + p3 + p4 + p1 +
  plot_layout(design = layout)

The other path is using `area()` inside `layout`, as shown below.

In [ ]:
layout <- c(
  area(t = 2, l = 1, b = 5, r = 4),
  area(t = 1, l = 3, b = 3, r = 5)
  )

p3 + p4 +
  plot_layout(design = layout)

Watch the specification here with `wrap_plots()`

In [ ]:
layout <- '
A##
#B#
##C
'
wrap_plots(A = p1, B = p2, C = p3, design = layout)

## Fixed-aspect plots

There are some plots that use fixed coordinates and these should not be disturbed. Here is an example

In [ ]:
library(urbnmapr)

ggplot() +
  geom_polygon(
    data = states,
    aes(
      x = long,
      y = lat,
      group = group,
      fill = state_abbv
      )
    ) +
  ggthemes::theme_map() +
  theme(legend.position = "none") +
  labs(title = "Fixed!!") -> mymap

mymap + p1 + p2 + p3

In [ ]:
ggplot() +
  geom_polygon(
    data = states,
    aes(
      x = long,
      y = lat,
      group = group,
      fill = state_abbv
      )
    ) +
  coord_fixed(1.3) +
  ggthemes::theme_map() +
  theme(legend.position = "none") +
  labs(title = "Fixed!!") -> mymap

mymap + p1 + p2 + p3

# Mapping
Maps are very powerful visualizations because they allow you to highlight and reflect patterns, clusters, with relative ease. For example, is poverty really higher in Appalachian counties? What about the percent of the population without health insurance? Literacy? Opioid deaths; do they follow transportation routes? What about COVID-19 cases? Maps to the rescue!

Building a map requires a few elements. First and foremost, you need some data to show on a map. Second, you need to have the geographic coordinates needed to build a map, basically the latitude and longitude of the geographies (states, cities, school districts, etc.) that you want to map. Third, you want a column that contains the names of the geographies you want to map, and these should be properly formatted (i.e., in titlecase) for displaying on the map.

Let us start by building a simple state map with the `{urbnmapr}` package. It comes with the necessary data for states and counties, respectively, and works well with `{ggplot2}`. Note the reliance on `geom_polygon()` now.

In [ ]:
glimpse(states)
glimpse(counties)

ggplot() +
  geom_polygon(data = states,
               aes(x = long, y = lat, group = group),
               fill = "white", color = "steelblue") +
  coord_fixed(1.3)

Note that this is just an empty map with the shapes of the states, and also that Alaska and Hawaii have been moved so that they can be displayed on the map. We could build a much better map by removing the x and y axis labels and tick marks, and setting a white background using `theme_map()` from the `{ggthemes}` package. We could also fill with some colors, say on the basis of the `state_name`.

In [ ]:
ggplot() +
  geom_polygon(data = states,
               aes(x = long, y = lat, group = group, fill = state_name),
               color = "white") +
  coord_fixed(1.3) +
  ggthemes::theme_map() +
  theme(legend.position = "none")

Okay, this is not very useful because it would be much better to color the map on the basis of some substantive variable. Let us see what lurks in the `statedata` file.

In [ ]:
glimpse(statedata)

Okay, two things stand out -- `horate` (the homeownership rate), and `medhhincome` (the median household income). Let us fill with median household income but to do so, we will need to join `statedata` to our `states` file. Why? Because we need coordinates to map anything and `statedata` does not contain coordinates. Then we can specify `fill = medhhincome` inside the `aes(...)` command.

In [ ]:
states %>%
  left_join(statedata, by = c("state_fips", "state_name")) -> state.df
glimpse(state.df)

In [ ]:
ggplot() +
  geom_polygon(data = state.df,
               aes(x = long, y = lat, group = group, fill = medhhincome),
               color = "white") +
  coord_fixed(1.3) +
  ggthemes::theme_map() +
  theme(legend.position = "bottom") +
  labs(
    title = "Median Household Income in the States (2015)",
    fill = ""
        ) +
  scale_fill_viridis_c(option = "magma")

What about working with counties? Sure, let us merge `countydata` with the `counties file and map.

In [ ]:
counties %>%
  left_join(countydata, by = c("county_fips")) -> county.df

ggplot() +
  geom_polygon(data = county.df,
               aes(x = long, y = lat, group = group, fill = medhhincome),
               color = "white", linewidth = 0.05) +
  coord_fixed(1.3) +
  ggthemes::theme_map() +
  theme(legend.position = "bottom") +
  labs(
    title = "Median Household Income in the Counties (2015)",
    fill = ""
        ) +
  scale_fill_viridis_c(option = "magma")

Maybe you are only interested in Florida?

In [ ]:
county.df %>%
  filter(state_abbv == "FL") %>%
  ggplot() +
  geom_polygon(
               aes(x = long, y = lat, group = group,
                   fill = medhhincome),
               color = "white", linewidth = 0.05
               ) +
  coord_fixed(1.3) +
  ggthemes::theme_map() +
  theme(legend.position = "bottom") +
  labs(
    title = "Median Household Income in Floria Counties (2015)",
    fill = ""
        ) +
  scale_fill_viridis_c(option = "plasma")

Hmm, so far so good but what if the data were for some geography not bundled with `{urbnmapr}`, school districts or places, for example? Not a problem, we just have to go the extra mile. First we would have to find, download, and upload the `shapefile`. Say I am looking for places (loosely described as municipalities) in New Hampshire. Well, the `{tigris}` package comes in handy because it allows you to get whatever geography's shapefiles you want. Below I am getting the shapefile for New Hampshire.

In [ ]:
library(tigris)

options(tigris_use_cache = TRUE)

In [ ]:
places(
  state = "NH",
  cb = TRUE,
  year = 2020,
  progress_bar = TRUE
  ) -> places.nh

In [ ]:
head(places.nh)

In [ ]:
ggplot() +
    geom_sf(
        data = places.nh,
        aes(
            fill = ALAND
            )
        )

Of course, the `fill` is superficial here. But say we had some data for places in New Hampshire, maybe the size of the population, or some other metric? Let us go and see what information we can grab at the place level from the CDC. We can then pull it down and filter just to NH, and then plot it.  

In [ ]:
load(here::here("data", "nh.data.RData"))

nh.df %>%
  left_join(nh.data, by = c("id" = "GEOID")) -> nh

glimpse(nh)

Now we plot:

In [ ]:
ggplot() +
  geom_polygon(
    data = subset(state.df, state_name == "New Hampshire"),
    aes(x = long, y = lat, group = group),
    fill = "white", color = "black"
    ) +
  geom_polygon(
    data = nh,
    aes(x = long, y = lat, group = group, fill = population)
    ) +
  coord_fixed(1.3) +
  scale_fill_viridis_c(option = "viridis") +
  ggthemes::theme_map() +
  theme(legend.position = "bottom") +
  labs(
    fill = "",
    title = "Population Distribution in New Hampshire's Places",
    subtitle = "(American Community Survey, 2014-2018)"
       )

You could have also filled by creating quartiles, etc., using `{Santoku}`, so do not forget that option.

Before we move on, one more map, to show you the possibilities. Here, I am
plotting the locations of the parking tickets issued in Philadelphia, this time with the `{leaflet}` package. First the tickets data-set, reduced to a random sample of 20% of the tickets issued in the month of December in 2017 (to keep the data size manageable). I have also created a `popup` that will display specific information if someone clicks on a point in the map.   

In [ ]:
readr::read_csv(
    "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-12-03/tickets.csv"
) %>%
  mutate(
    year = lubridate::year(issue_datetime),
    month = lubridate::month(issue_datetime)
         ) %>%
  filter(month == 12, lon > -75.5) %>%
  sample_frac(0.2) -> tickets

tickets %>%
  unite(display, c(issuing_agency, issue_datetime, fine),
       sep = "; ", remove = FALSE) -> tickets

In [ ]:
library(leaflet)
library(htmltools)
library(widgetframe)

In [ ]:
leaflet(tickets) %>%
  addTiles() %>%
  addCircles(lng = ~ lon, lat = ~ lat, popup = ~htmlEscape(display),
             color = "steelblue", opacity = 0.10) 

Voila! A few lines of code and we have an interactive map that can be used to display whatever evidence we want to display. Note that you need geographic coordinates since without them the data cannot be attached to a physical location.  

Let us see another twist on this. Say I am trying to map the total number of COVID-19 cases in Ohio. I know the county and I know the number of cases that occurred, as well as the latitude and longitude of each county. Well, I can build a similar plot except making the size of the circle conditional upon the number of cases. The more the cases, the larger the radius of the circle.

In [ ]:
readr::read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv") -> covid

covid %>%
  filter(state == "Ohio", date == "2020-04-17") -> cov19

But there are no coordinates in `cov19`. Well, I can get those from the `{housingData}` package, and then merge that with `cov19`.

In [ ]:
install.packages("housingData")

In [ ]:
library(housingData)

In [ ]:
geoCounty %>%
  filter(state == "OH") %>%
  separate(county,
           into = c("countyname", "extra"),
           sep = " County",
           remove = TRUE) %>%
  mutate(countyname = stringr::str_to_sentence(countyname)) -> oh

oh %>%
  left_join(
      cov19, 
      by = c(
          "countyname" = "county",
          "state" = "state",
          "fips" = "fips"
      )
  ) -> ohcov19

ohcov19 %>%
  unite(display, c(countyname, cases),
        sep = ": ",
        remove = FALSE) -> ohcov19

glimpse(ohcov19)

Okay, now we have everything we need to build the map.

In [ ]:
leaflet(ohcov19) %>%
  addTiles() %>%
  addCircleMarkers(
      lng = ~ lon, 
      lat = ~ lat, 
      popup = ~htmlEscape(display),
      color = "steelblue", 
      opacity = 0.50, 
      radius = ~sqrt(cases)
    ) 

Now here is a county-level chart that shows the total number of cases as of April 14, 2020. This is stored in `tab3` created as shown below. Pay attention to this creation because we are not just creating a frequency table but also adding in a specific key we are calling `code` because we will need to join these data to the map data.